In [1]:
import sys
sys.path.append('/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4')
sys.path.append('/kaggle/input/for-vesuvius/EfficientNet-PyTorch-master/EfficientNet-PyTorch-master')
sys.path.append('/kaggle/input/for-vesuvius/pytorch-image-models-main/pytorch-image-models-main')
sys.path.append('/kaggle/input/for-vesuvius/segmentation_models.pytorch-master/segmentation_models.pytorch-master')
sys.path.append('/kaggle/input/for-vesuvius/wandb-main/wandb-main')
sys.path.append('/kaggle/input/for-vesuvius/pytorch-gradual-warmup-lr-master/pytorch-gradual-warmup-lr-master')

# Imports

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss
import matplotlib.pyplot as plt
import sys
import os
import gc
import sys
import pickle
import warnings
import math
import time
import random
import argparse
import importlib
from tqdm import tqdm
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam, SGD, AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import segmentation_models_pytorch as smp
from warmup_scheduler import GradualWarmupScheduler
import cv2

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
import datetime

import wandb
wandb.login(key = '68a8aa0d16f78ed8a8b3f6d0255850bdbf2f7b61')

/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels/models/dpn.py:255: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if block_type is 'proj':
/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels/models/dpn.py:258: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif block_type is 'down':
/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4/pretrainedmodels/models/dpn.py:262: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert block_type is 'normal'
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# sys.path.append('/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4')
# sys.path.append('/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch-master')
# sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
# sys.path.append('/kaggle/input/segmentation-models-pytorch/segmentation_models.pytorch-master')

# Main Configuration Class

In [5]:
class CFG:
    # ============== comp exp name =============
    comp_name = 'vesuvius'
    exp_name = 'Model_testing'
    comp_dir_path = '/kaggle/input/'
    comp_folder_name = 'vesuvius-challenge-ink-detection'
    comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'

    # ============== pred target =============
    target_size = 1

    # ============== model cfg =============
    model_name = 'Unet'
    backbone = 'timm-resnest200e' #'se_resnext50_32x4d'

    in_chans = 6 # 65
    # ============== training cfg =============
    size = 384
    tile_size = 384
    stride = tile_size // 2

    train_batch_size = 16 # 32
    valid_batch_size = train_batch_size * 2
    use_amp = True

    scheduler = 'GradualWarmupSchedulerV2' # 'GradualWarmupSchedulerV2' # 'CosineAnnealingLR'
    epochs = 60 # 30

    # adamW warmupあり
    warmup_factor = 10
    # lr = 1e-4 / warmup_factor
    lr = 1e-4 / warmup_factor

    # ============== fold =============
    valid_id = [1]

    # objective_cv = 'binary'  # 'binary', 'multiclass', 'regression'
    metric_direction = 'maximize'  # maximize, 'minimize'
    # metrics = 'dice_coef'

    # ============== fixed =============
    pretrained = True
    inf_weight = 'best'  # 'best'
    loss = "BCE"

    min_lr = 1e-7
    weight_decay = 0
    max_grad_norm = 1000

    print_freq = 50
    num_workers = 2

    seed = 310
    
    # ============== set dataset path =============
    print('set dataset path')

    outputs_path = f'/kaggle/working/outputs/{comp_name}/{exp_name}/'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path + \
        f'{comp_name}-models/'

    figures_dir = outputs_path + 'figures/'

    log_dir = outputs_path + 'logs/'
    log_path = log_dir + f'{exp_name}.txt'

    train_aug_list = [
    #A.RandomCrop(CFG.tile_size,CFG.tile_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.SafeRotate(p = 0.3),
    A.Transpose(p=0.5),
    A.CoarseDropout(max_holes=8, max_height=int(size*0.1), max_width=int(size*0.1), min_holes=3, fill_value=0, mask_fill_value=0, p=0.5),
    A.ElasticTransform(p = 0.3),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    A.Emboss(p = 0.3),
    A.RandomBrightnessContrast(p=0.4),
    A.RandomGridShuffle(grid=(4, 4), p = 0.4),
    A.RingingOvershoot(p = 0.2),
    #A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.GaussianBlur(p = 0.5),
    A.Sharpen(p = 0.2),
    A.Normalize(
        mean= [0] * in_chans,
        std= [1] * in_chans,
    ),
        ToTensorV2(transpose_mask=True),
 ]

    valid_aug_list = [
        A.Resize(size, size),
        A.Normalize(
            mean= [0] * in_chans,
            std= [1] * in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]


set dataset path


## Helper Functions (as usual)

In [6]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
def init_logger(log_file):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def set_seed(seed=None, cudnn_deterministic=True):
    if seed is None:
        seed = 310
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = cudnn_deterministic
    torch.backends.cudnn.benchmark = False

In [8]:
def make_dirs(cfg):
    for dir in [cfg.model_dir, cfg.figures_dir, cfg.submission_dir, cfg.log_dir]:
        os.makedirs(dir, exist_ok=True)

In [9]:
def make_folder(path):
        os.makedirs(path, exist_ok=True)

In [10]:
def cfg_init(cfg, mode='train'):
    set_seed(cfg.seed)

    if mode == 'train':
        make_dirs(cfg)

In [11]:
cfg_init(CFG)

Logger = init_logger(log_file=CFG.log_path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Image and Mask (by mask, it means the inklabels.png)

### The mask will act as the label for segmentation.

In [12]:
def read_image_mask(fragment_id):

    images = []

    for i in tqdm(range(6)):
        image = np.load('/kaggle/input/oof-vesuvius-predictions/'+f'{i}_{fragment_id}.npy').astype(np.uint8)

        pad0 = (CFG.tile_size - image.shape[0] % CFG.tile_size)
        pad1 = (CFG.tile_size - image.shape[1] % CFG.tile_size)

        image = np.pad(image, [(0, pad0), (0, pad1)], constant_values=0)

        images.append(image)
    images = np.stack(images, axis=2)

    mask = cv2.imread(CFG.comp_dataset_path + f"train/{fragment_id}/mask.png", 0)
    label = cv2.imread(CFG.comp_dataset_path +  f"train/{fragment_id}/inklabels.png",0)
    label = np.pad(label, [(0, pad0), (0, pad1)], constant_values=0)
    mask = (np.pad(mask, [(0, pad0), (0, pad1)], constant_values=0)/255)
    label = label.astype('float32')
    label /= 255.0
    
    return images, mask , label

In [13]:
def get_train_valid_dataset(valid_id):
    train_images = []
    train_masks = []

    valid_images = []
    valid_masks = []
    valid_xyxys = []

    for fragment_id in range(1, 4):

        image, mask, label = read_image_mask(fragment_id)

        x1_list = list(range(0, image.shape[1]-CFG.tile_size+1, CFG.stride))
        y1_list = list(range(0, image.shape[0]-CFG.tile_size+1, CFG.stride))

        for y1 in y1_list:
            for x1 in x1_list:
                y2 = y1 + CFG.tile_size
                x2 = x1 + CFG.tile_size
                # xyxys.append((x1, y1, x2, y2))
        
                if fragment_id == valid_id:
                    valid_images.append(image[y1:y2, x1:x2])
                    valid_masks.append(label[y1:y2, x1:x2, None])

                    valid_xyxys.append([x1, y1, x2, y2])
                else:
                    if np.sum(mask[y1:y2, x1:x2]) == CFG.tile_size**2:
                    #if True:
                        train_images.append(image[y1:y2, x1:x2])
                        train_masks.append(label[y1:y2, x1:x2, None])

    return train_images, train_masks, valid_images, valid_masks, valid_xyxys

# Dataset Classes

In [14]:
def get_transforms(data, cfg):
    if data == 'train':
        aug = A.Compose(cfg.train_aug_list)
    elif data == 'valid':
        aug = A.Compose(cfg.valid_aug_list)
    return aug

class CustomDataset(Dataset):
    def __init__(self, images, cfg, labels=None, transform=None):
        self.images = images
        self.cfg = cfg
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            data = self.transform(image=image, mask=label)
            image = data['image']
            label = data['mask']

        return image, label

## Model (finally the good stuff)

In [15]:
class CustomModel(nn.Module):
    def __init__(self, cfg, weight=None):
        super().__init__()
        self.cfg = cfg

        self.encoder = smp.Unet(
            encoder_name=cfg.backbone, 
            encoder_weights=weight,
            in_channels=cfg.in_chans,
            classes=cfg.target_size,
            activation=None,
        )

    def forward(self, image):
        output = self.encoder(image)
        # output = output.squeeze(-1)
        return output

In [16]:
def build_model(cfg, weight="imagenet"):
    print('model_name', cfg.model_name)
    print('backbone', cfg.backbone)

    model = CustomModel(cfg, weight)

    return model

## Scheduler

In [17]:
torch.optim.lr_scheduler.ReduceLROnPlateau
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    """
    https://www.kaggle.com/code/underwearfitting/single-fold-training-of-resnet200d-lb0-965
    """
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(
            optimizer, multiplier, total_epoch, after_scheduler)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [
                        base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

def get_scheduler(cfg, optimizer,after = 1):
    
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, 15, eta_min=CFG.min_lr)
    
    #seheduler_reducelron = ReduceLROnPlateau(optimizer,factor = 0.1,patience = 2,threshold = 0.01,min_lr = CFG.min_lr)
    
    scheduler = GradualWarmupSchedulerV2(
        optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

    return scheduler

def scheduler_step(scheduler, avg_val_loss, epoch):
    scheduler.step(metrics  = avg_val_loss,epoch = epoch)


## Initializing the Model, optimizer and the scheduler

In [18]:
# model = build_model(CFG)
# model.to(device)

# optimizer = AdamW(model.parameters(), lr=CFG.lr)
# scheduler = get_scheduler(CFG, optimizer)

In [19]:
# class FocalLoss(nn.Module):
    
#     def __init__(self,alpha = 0.25, gamma = 2):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma
        
#     def forward(self,pred_logits,target):
#         pred = pred_logits.sigmoid()
#         ce = F.binary_cross_entropy_with_logits(pred_logits, target, reduction = 'none')
#         alpha = target * self.alpha + (1.0-target) * (1.0 - self.alpha)
#         pt = torch.where(target == 1, pred, 1-pred)
#         return (alpha * (1.0 - pt) ** self.gamma * ce).mean()

## Losses

In [20]:
# alpha = 0.5
# beta = 1 - alpha
# FLoss = FocalLoss(alpha = 0.80, gamma=2)
# DiceLoss = smp.losses.DiceLoss(mode='binary')
# BCELoss = smp.losses.SoftBCEWithLogitsLoss()
# TverskyLoss = smp.losses.TverskyLoss(mode='binary', log_loss=False, alpha=alpha, beta=beta)

In [21]:
# def criterion(y_pred, y_true):
#     if CFG.loss = 'BCE': 
#         return 0.5 * BCELoss(y_pred, y_true) + 0.5 * DiceLoss(y_pred, y_true)
#     elif CFG.loss = 'FL:
#         return 0.5 * FLoss(torch.sigmoid(y_pred), y_true) + 0.5 * DiceLoss(y_pred, y_true)
#     return DiceLoss(y_pred, y_true)
    # return 0.5 * BCELoss(y_pred, y_true) + 0.5 * TverskyLoss(y_pred, y_true)
#     return BCELoss(y_pred, y_true)

In [22]:
DiceLoss = smp.losses.DiceLoss(mode='binary')
BCELoss = smp.losses.SoftBCEWithLogitsLoss()
def criterion(y_pred, y_true):
    if CFG.loss == 'BCE': 
        return 0.5 * BCELoss(y_pred, y_true) + 0.5 * DiceLoss(y_pred, y_true)
    elif CFG.loss == 'FL':
        return 0.5 * FLoss(y_pred, y_true) + 0.5 * DiceLoss(y_pred, y_true)

## Training and Validation Function

In [23]:
def train_fn(train_loader, model, criterion, optimizer, device):
    model.train()

    scaler = GradScaler(enabled=CFG.use_amp)
    losses = AverageMeter()

    for step, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        with autocast(CFG.use_amp):
            y_preds = model(images)
            loss = criterion(y_preds, labels)

        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        
#         scaler.unscale_(optimizer)
#         torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)

        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), CFG.max_grad_norm)

        scaler.step(optimizer)
        scaler.update()
        
        del images, labels
        garbage = gc.collect()
        torch.cuda.empty_cache()       

    return losses.avg

def valid_fn(valid_loader, model, criterion, device, valid_xyxys, valid_mask_gt):
    mask_pred = np.zeros(valid_mask_gt.shape)
    mask_count = np.zeros(valid_mask_gt.shape)

    model.eval()
    losses = AverageMeter()

    for step, (images, labels) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        # make whole mask
        y_preds = torch.sigmoid(y_preds).to('cpu').numpy()
        start_idx = step*CFG.valid_batch_size
        end_idx = start_idx + batch_size
        for i, (x1, y1, x2, y2) in enumerate(valid_xyxys[start_idx:end_idx]):
            mask_pred[y1:y2, x1:x2] += y_preds[i].squeeze(0)
            mask_count[y1:y2, x1:x2] += np.ones((CFG.tile_size, CFG.tile_size))
            
        del images, labels
        garbage = gc.collect()
        torch.cuda.empty_cache() 

    print(f'mask_count_min: {mask_count.min()}')
    mask_pred /= mask_count
     
    return losses.avg, mask_pred

## Metrics for the competition

In [24]:
from sklearn.metrics import fbeta_score

def fbeta_numpy(targets, preds, beta=0.5, smooth=1e-5):
    """
    https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection/discussion/397288
    """
    y_true_count = targets.sum()
    ctp = preds[targets==1].sum()
    cfp = preds[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)
    
    del targets, preds
    garbage = gc.collect()
    return dice

def calc_fbeta(mask, mask_pred):
    mask = mask.astype(np.uint8).flatten()
    mask_pred = mask_pred.flatten()

    best_th = 0
    best_dice = 0
    for th in np.array(range(100, 950+1, 25)) / 1000:
        
        # dice = fbeta_score(mask, (mask_pred >= th).astype(int), beta=0.5)
        dice = fbeta_numpy(mask, (mask_pred >= th).astype(np.uint8), beta=0.5)
        print(f'th: {th}, fbeta: {dice}')

        if dice > best_dice:
            best_dice = dice
            best_th = th
    del mask, mask_pred
    garbage = gc.collect()
    
    
    return best_dice, best_th


def calc_cv(mask_gt, mask_pred):
    best_dice, best_th = calc_fbeta(mask_gt, mask_pred)

    return best_dice, best_th

## Main

In [25]:
def train(train_images, train_masks, valid_images, valid_masks, valid_xyxys):
    for fold in CFG.valid_id:
        valid_xyxys = np.stack(valid_xyxys)


        train_dataset = CustomDataset(train_images, CFG, labels=train_masks,
                                      transform=get_transforms(data='train', cfg=CFG))

        valid_dataset = CustomDataset(valid_images, CFG, labels=valid_masks,
                                      transform=get_transforms(data='valid', cfg=CFG))

        train_loader = DataLoader(train_dataset,
                                  batch_size=CFG.train_batch_size,
                                  shuffle=True,
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                                  )

        valid_loader = DataLoader(valid_dataset,
                                  batch_size=CFG.valid_batch_size,
                                  shuffle=False,
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=False)


        valid_mask_gt = cv2.imread(CFG.comp_dataset_path + f"train/{fold}/inklabels.png", 0)
        valid_mask_gt = valid_mask_gt / 255
        pad0 = (CFG.tile_size - valid_mask_gt.shape[0] % CFG.tile_size)
        pad1 = (CFG.tile_size - valid_mask_gt.shape[1] % CFG.tile_size)
        valid_mask_gt = np.pad(valid_mask_gt, [(0, pad0), (0, pad1)], constant_values=0)


        if CFG.metric_direction == 'minimize':
            best_score = np.inf
        elif CFG.metric_direction == 'maximize':
            best_score = -1

        best_loss = np.inf
        
        Logger.info(f'\n {CFG.backbone} \n')

        model = build_model(CFG)
        model.to(device)

        optimizer = AdamW(model.parameters(), lr=CFG.lr)
        scheduler = get_scheduler(CFG, optimizer)
        model_code = random.randint(1,100000)
        run = wandb.init(project=f"Stacking", config=config, name = f"Unet timm-resenst200e Fold 1 {model_code}")
        
        patience_break = 0
        
        for epoch in range(CFG.epochs):
            start_time = time.time()

            # train
            avg_loss = train_fn(train_loader, model, criterion, optimizer, device)

            # eval
            avg_val_loss, mask_pred = valid_fn(
                valid_loader, model, criterion, device, valid_xyxys, valid_mask_gt)

            scheduler_step(scheduler, avg_val_loss, epoch)

            best_dice, best_th = calc_cv(valid_mask_gt, mask_pred)

            score = best_dice

            elapsed = time.time() - start_time
            
            wandb.log({"Train_loss": avg_loss, "Val_los": avg_val_loss, "Score": score, "Epoch":epoch+1, 'th':best_th, 'Time':elapsed})

            
            Logger.info(
                f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f} avgScore: {score:.4f} th: {best_th} time: {elapsed:.0f}s')

            if CFG.metric_direction == 'minimize':
                update_best = score < best_score
            elif CFG.metric_direction == 'maximize':
                update_best = score > best_score

            if update_best:
                patience_break = 0
                best_loss = avg_val_loss
                best_score = score
                
                best_save = {'Epoch':epoch+1,'Train_loss':avg_loss,'Val_loss':avg_val_loss,'Score':score,'Best_th':best_th}

                torch.save({'model': model.state_dict()},
                            CFG.model_dir + f'BEST.pth')
                
            else:
                patience_break += 1
                if patience_break == 10:
                    break
            #torch.save({'model': model.state_dict()},
                       #CFG.model_dir + f'{CFG.model_name}_{CFG.backbone}_Epoch{epoch+1}_avgScore{score:.4f}_th{best_th}_fold2_{model_code}.pth')

            torch.cuda.empty_cache()
            abd = gc.collect()
            
        wandb.log({"Best_Train_loss": best_save['Train_loss'], "Best_Val_los": best_save['Val_loss'], 
                   "Best_Score": best_save['Score'], "Best_Epoch":best_save['Epoch'], 
                   'Best_th':best_save['Best_th']})
        
        Logger.info(
            f'''\n Model fold 3: {config['model']} Best_Epoch {best_save['Epoch']} - Best_train_loss: {best_save['Train_loss']}  
            Best_val_loss: {best_save['Val_loss']} Best_Score: {best_save['Score']} 
            Best_th: {best_save['Best_th']}''')
        run.finish()
        del model
        gc.collect()
        del optimizer
        del scheduler
        garbage = gc.collect()
        torch.cuda.empty_cache()

In [26]:
config = {
    'arch' : 'Unet',
    'model' :CFG.backbone,
    'fold': 1,
    'version' : 'KU5_SF_V1',
}

In [27]:
# def update_CFG(config,CFG):
#     CFG.model_name = config['arch']
#     CFG.backbone = 'mit_b2'


In [28]:
train_images, train_masks, valid_images, valid_masks, valid_xyxys = get_train_valid_dataset(CFG.valid_id[0])
# train(train_images, train_masks, valid_images, valid_masks, valid_xyxys)

100%|██████████| 6/6 [00:01<00:00,  4.22it/s]


In [29]:
train(train_images, train_masks, valid_images, valid_masks, valid_xyxys)


 timm-resnest200e 



model_name Unet
backbone timm-resnest200e


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-resnest/resnest200-75117900.pth" to /root/.cache/torch/hub/checkpoints/resnest200-75117900.pth
100%|██████████| 269M/269M [00:00<00:00, 292MB/s]
wandb: Currently logged in as: whybeaking (i-am-noob). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230614_122307-6m32yzxz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Unet timm-resenst200e Fold 1 52281
wandb: ⭐️ View project at https://wandb.ai/i-am-noob/Stacking
wandb: 🚀 View run at https://wandb.ai/i-am-noob/Stacking/runs/6m32yzxz
100%|██████████| 45/45 [00:52<00:00,  1.17s/it]
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary a

mask_count_min: 1.0
th: 0.1, fbeta: 0.1181725139113061
th: 0.125, fbeta: 0.11820914417458282
th: 0.15, fbeta: 0.11822211910874526
th: 0.175, fbeta: 0.11866569311155006
th: 0.2, fbeta: 0.12024240330880116
th: 0.225, fbeta: 0.12353305156957953
th: 0.25, fbeta: 0.12859882491104732
th: 0.275, fbeta: 0.16815517582634798
th: 0.3, fbeta: 0.26375718391734093
th: 0.325, fbeta: 0.38258891996334154
th: 0.35, fbeta: 0.45150019401652924
th: 0.375, fbeta: 0.48925593460156225
th: 0.4, fbeta: 0.517375847946141
th: 0.425, fbeta: 0.5306670437306722
th: 0.45, fbeta: 0.5378532654826947
th: 0.475, fbeta: 0.54134882476674
th: 0.5, fbeta: 0.5423852549790815
th: 0.525, fbeta: 0.5409857957383819
th: 0.55, fbeta: 0.5374135910719551
th: 0.575, fbeta: 0.5313399555553006
th: 0.6, fbeta: 0.5228851975862548
th: 0.625, fbeta: 0.512333745980145
th: 0.65, fbeta: 0.49977774695934307
th: 0.675, fbeta: 0.48500735237656534
th: 0.7, fbeta: 0.4680639945704758
th: 0.725, fbeta: 0.4488027851560429
th: 0.75, fbeta: 0.4265301516

Epoch 1 - avg_train_loss: 0.6569  avg_val_loss: 0.5745 avgScore: 0.5424 th: 0.5 time: 450s
100%|██████████| 45/45 [00:51<00:00,  1.14s/it]


...

Epoch 19 - avg_train_loss: 0.5195  avg_val_loss: 0.4487 avgScore: 0.6109 th: 0.7 time: 436s
100%|██████████| 45/45 [00:51<00:00,  1.14s/it]


mask_count_min: 1.0
th: 0.1, fbeta: 0.13787648028519314
th: 0.125, fbeta: 0.38582308863643533
th: 0.15, fbeta: 0.4833603778849866
th: 0.175, fbeta: 0.5038436226079984
th: 0.2, fbeta: 0.5191748493498183
th: 0.225, fbeta: 0.5314540048214558
th: 0.25, fbeta: 0.5420537743930542
th: 0.275, fbeta: 0.5515466275664055
th: 0.3, fbeta: 0.5603752099276992
th: 0.325, fbeta: 0.5681872131602389
th: 0.35, fbeta: 0.575134252785202
th: 0.375, fbeta: 0.5813880411568719
th: 0.4, fbeta: 0.586808244409554
th: 0.425, fbeta: 0.5915419576861407
th: 0.45, fbeta: 0.5958267210894784
th: 0.475, fbeta: 0.5994592651108934
th: 0.5, fbeta: 0.6026604552310995
th: 0.525, fbeta: 0.6057380105785748
th: 0.55, fbeta: 0.6089399477850926
th: 0.575, fbeta: 0.6113415967638304
th: 0.6, fbeta: 0.612514232464999
th: 0.625, fbeta: 0.6127938282162757
th: 0.65, fbeta: 0.612844408992159
th: 0.675, fbeta: 0.6132199796128543
th: 0.7, fbeta: 0.6132561686285398
th: 0.725, fbeta: 0.6128572157478464
th: 0.75, fbeta: 0.611621632829115
th: 0

Epoch 20 - avg_train_loss: 0.5213  avg_val_loss: 0.4469 avgScore: 0.6133 th: 0.7 time: 443s
100%|██████████| 45/45 [00:51<00:00,  1.15s/it]


mask_count_min: 1.0
th: 0.1, fbeta: 0.13264596042694193
th: 0.125, fbeta: 0.22464179649455787
th: 0.15, fbeta: 0.4651208271127908
th: 0.175, fbeta: 0.49039716318668
th: 0.2, fbeta: 0.5070139043833624
th: 0.225, fbeta: 0.5203453382812162
th: 0.25, fbeta: 0.5314695578274696
th: 0.275, fbeta: 0.5411060061078364
th: 0.3, fbeta: 0.549734257246959
th: 0.325, fbeta: 0.5575685795434403
th: 0.35, fbeta: 0.564678058412879
th: 0.375, fbeta: 0.5704336158153311
th: 0.4, fbeta: 0.5753647598604531
th: 0.425, fbeta: 0.5796871990992974
th: 0.45, fbeta: 0.5837773267756526
th: 0.475, fbeta: 0.587399921325241
th: 0.5, fbeta: 0.5901158338228699
th: 0.525, fbeta: 0.5924883599426916
th: 0.55, fbeta: 0.5944750485188465
th: 0.575, fbeta: 0.5965784205421389
th: 0.6, fbeta: 0.5983005582803289
th: 0.625, fbeta: 0.5993497976020343
th: 0.65, fbeta: 0.6000441412658388
th: 0.675, fbeta: 0.6007138340953275
th: 0.7, fbeta: 0.6009767412629305
th: 0.725, fbeta: 0.6001208590911289
th: 0.75, fbeta: 0.5984053019655909
th: 0

Epoch 21 - avg_train_loss: 0.5245  avg_val_loss: 0.4560 avgScore: 0.6010 th: 0.7 time: 438s

 Model fold 3: timm-resnest200e Best_Epoch 11 - Best_train_loss: 0.5289697703388002  
            Best_val_loss: 0.44957822007852344 Best_Score: 0.6148207261861526 
            Best_th: 0.725
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:      Best_Epoch ▁
wandb:      Best_Score ▁
wandb: Best_Train_loss ▁
wandb:    Best_Val_los ▁
wandb:         Best_th ▁
wandb:           Epoch ▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
wandb:           Score ▁▄▄▇▆▇▇████▇▇▇██▇▇██▇
wandb:            Time █▂▃▂▁▄▃▅▃▂▁▂▅▂▁▂▃▄▄▆▄
wandb:      Train_loss █▆▄▃▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁
wandb:         Val_los █▇▃▁▂▂▁▃▁▂▂▁▂▁▁▁▂▁▁▁▂
wandb:              th ▁▂▂▇██▆▆▄▆▆▇▇▆▆▅▅▆▆▆▆
wandb: 
wandb: Run summary:
wandb:      Best_Epoch 11
wandb:      Best_Score 0.61482
wandb: Best_Train_loss 0.52897
wandb:    Best_Val_los 0.44958
wandb:         Best_th 0.725
wandb:           Epoch 21
wandb:           Score 0.60098
wand

In [30]:
# for modell in model_to_try:
#     for chans in chans_to_try:
#         config['in_chans'] = chans
#         update_CFG(config,CFG)
#         train_images, train_masks, valid_images, valid_masks, valid_xyxys = get_train_valid_dataset(1)
#         CFG.train_aug_list[-2] = A.Normalize(
#                                     mean= [0] * chans,
#                                     std= [1] * chans,
#                                         )
#         CFG.valid_aug_list[-2] = A.Normalize(
#                                     mean= [0] * chans,
#                                     std= [1] * chans,
#                                         ) 
#         train(train_images, train_masks, valid_images, valid_masks, valid_xyxys)
#         abd = gc.collect()
#         del train_images[:], train_masks[:], valid_images[:], valid_masks[:], valid_xyxys[:]
#         del train_images, train_masks, valid_images, valid_masks, valid_xyxys
#         abd = gc.collect()
#         torch.cuda.empty_cache()

In [31]:
# train_images, train_masks, valid_images, valid_masks, valid_xyxys = get_train_valid_dataset(1)